In [22]:
import carla
import random

def draw_waypoint_pairs(world, waypoint_pairs):
    for index, (wp1, wp2) in enumerate(waypoint_pairs):
        # Calcola il numero progressivo della coppia
        pair_number = index + 1

        print(f"indice {index}: {wp1} - {wp2}")

        # Disegna una 'X' su wp1 con il numero progressivo
        world.debug.draw_string(wp1.transform.location, f"X {pair_number}", 
                                draw_shadow=True, 
                                color=carla.Color(r=0, g=255, b=0), 
                                life_time=60.0, 
                                persistent_lines=True)

        # Disegna una 'X' su wp2 con lo stesso numero progressivo
        world.debug.draw_string(wp2.transform.location, f"X {pair_number}", 
                                draw_shadow=True, 
                                color=carla.Color(r=255, g=0, b=0), 
                                life_time=60.0, 
                                persistent_lines=True)

# Connessione a Carla
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
map = world.get_map()

# Trova un incrocio casuale
spawn_points = map.get_spawn_points()
random_point = random.choice(spawn_points)
waypoint = map.get_waypoint(random_point.location)

# Cerca un incrocio a caso nella mappa a partire da un pnto di spawn a caso
while waypoint.get_junction() is None:
    random_point = random.choice(spawn_points)
    waypoint = map.get_waypoint(random_point.location)

#il problema ritorna le coppie di wp dei segmenti di strada coinvolti nell'incrocio
#ovviamente alcuni estremi di coppia possono essere in comune dato che da un punto puoi andare a destra o doritto
#inoltre non è detto che questi waypoints siano punti di spawn.. quindi bisogna pure capire deventualmente dove far uscire l'auto
draw_waypoint_pairs(world, waypoint.get_junction().get_waypoints(carla.LaneType.Driving))

# Posiziona la telecamera del simulatore sopra l'incrocio
spectator = world.get_spectator()
camera_location = carla.Location(x=waypoint.transform.location.x, y=waypoint.transform.location.y, z=50)
camera_rotation = carla.Rotation(pitch=-90)  # Orientamento verso il basso
spectator.set_transform(carla.Transform(camera_location, camera_rotation))



indice 0: Waypoint(Transform(Location(x=-41.573959, y=82.625427, z=0.000000), Rotation(pitch=0.000000, yaw=-90.161232, roll=0.000000))) - Waypoint(Transform(Location(x=-27.022135, y=69.714012, z=0.000000), Rotation(pitch=0.000000, yaw=-359.926727, roll=0.000000)))
indice 1: Waypoint(Transform(Location(x=-48.660942, y=51.725246, z=0.000000), Rotation(pitch=360.000000, yaw=89.838768, roll=0.000000))) - Waypoint(Transform(Location(x=-48.573933, y=82.645119, z=0.000000), Rotation(pitch=360.000000, yaw=89.838768, roll=0.000000)))
indice 2: Waypoint(Transform(Location(x=-52.160931, y=51.735096, z=0.000000), Rotation(pitch=360.000000, yaw=89.838768, roll=0.000000))) - Waypoint(Transform(Location(x=-52.073921, y=82.654968, z=0.000000), Rotation(pitch=360.000000, yaw=89.838768, roll=0.000000)))
indice 3: Waypoint(Transform(Location(x=-41.573959, y=82.625427, z=0.000000), Rotation(pitch=0.000000, yaw=-90.161232, roll=0.000000))) - Waypoint(Transform(Location(x=-41.660969, y=51.705547, z=0.000000